In [67]:
import ujson
import csv
import pandas as pd
import numpy as np
import gzip


In dit notebook volgt een analyse van alle publicaties van op officiele bekendmakingen van eerst de waterschappen, maar later mogelijk ook nog de provincies en de gemeenten. Hierbij worden de volgende vragen beantwoord:

- In welke verschillende informatietypen kunnen de publicaties worden ingedeeld. (De rest van de analyses worden dan met slechts 1 van deze informatietypen gedaan. Hoofdzakelijk de beschikkingen.
- Hoeveel publicaties hebben de verschillende organisaties binnen dit type door de jaren heen gedaan?
- Hoeveel gebruik maken de verschillende organisaties van het publiceren van bijlagen bij de publicaties?
- Welke geografische metadata is beschikbaar?
- Welke rubrieken zijn er precies verzameld en kunnen deze gekoppeld worden aan een eigen TOOI code?
- Welke topics zijn er precies verzameld?

Verder wordt er ook nog ingegaan op extreme outliers en w

In [68]:
gzipped_file = "SFTP_beschikkingen_wsb.json.gz"
# gzipped_file = "SFTP_beschikkingen_prb.json.gz"

with gzip.open(gzipped_file, "rt", encoding="utf-8") as file:
    beschikkingen = ujson.load(file)

In [69]:
# Tel alle keren dat een bepaald GEOattribuut voorkomt zodat we ook hiermee kunnen analyseren

alle_beschikkingen = {}
counter = 0
for k,v in beschikkingen.items():
    for c, i in enumerate(v):
        doc = i['infobox']
        if not isinstance(beschikkingen[k][c]['infobox']['foi_files'], int):
            beschikkingen[k][c]['infobox']['foi_files'] = len(doc['foi_files']['foi_documenten'])
        if 'foi_overheidOP_geo' in doc:
            if doc['foi_overheidOP_geo'] != []:
                count_dict = {}
                for i in doc['foi_overheidOP_geo']:
                    for KEY,VALUE in i.items():
                        if KEY not in count_dict.keys():
                            count_dict[KEY] = 1
                        else:
                            count_dict[KEY]+=1
    
                for KEY2,VALUE2 in count_dict.items():
                    beschikkingen[k][c]['infobox'][KEY2] = VALUE2      
                        
            beschikkingen[k][c]['infobox'].pop('foi_overheidOP_geo')

        alle_beschikkingen[counter] = beschikkingen[k][c]['infobox']
        counter+=1

In [70]:
df = pd.DataFrame(alle_beschikkingen).T

In [71]:
display(df.head())

,dc_title,dc_date_year,foi_publishedDate,dc_source,tooiwl_topic,dc_publisher_name,foi_startDate,foi_endDate,tooiwl_rubriek,dc_type,...,OVERHEIDop.huisletter,kml_LineString,OVERHEIDop.geometrieLabel,kml_Polygon,OVERHEIDop.waterschapsnaam,OVERHEIDop.huisnummerToevoeging,OVERHEIDop.effectgebied,Test_Terinlagelegging,OVERHEIDop.kadastraleSectie,OVERHEIDop.perceelnummer
0,Aanvraag watervergunning vernieuwen remmingswe...,2013,05-03-2013,https://zoek.officielebekendmakingen.nl/wsb-20...,Natuur en milieu | Water,Waterschap Brabantse Delta,2013-02-28,2013-03-28,Beschikkingen | aanvraag,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Verleende vergunning 13UT002188 ingevolge de K...,2013,2013-03-11,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-02-26,2013-04-09,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Verleende vergunning 13UT006559 ingevolge de K...,2013,2013-06-13,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-05-21,2013-07-02,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Verleende vergunning 13UT006525 ingevolge de K...,2013,2013-06-13,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-05-21,2013-07-02,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Verleende vergunning 13UT006589 ingevolge de K...,2013,13-06-2013,https://zoek.officielebekendmakingen.nl/wsb-20...,Ruimte en infrastructuur | Waterkeringen en wa...,Waterschap Brabantse Delta,2013-05-22,2013-07-03,Beschikkingen | afhandeling,2k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
def rubrieken(k):
    if 'beschikking' in k.lower() or 'vergunning' in k.lower():
        return 'Beschikking'
    elif k in ['algemeen verbindend voorschrift (verordening)', 'Overige besluiten van algemene strekking', 'beleidsregel','Beleidsregel', 'Beleidsregels', 'delegatie- of mandaatbesluit','bekendmaking aan de scheepvaart', 'algemeenverbindendverklaring van cao-bepalingen', 'gemeenschappelijke regeling', 'ruimtelijk plan of omgevingsdocument','verkeersbesluit of -mededeling', 'ander besluit van algemene strekking']:
        return 'Besluit van Algemene Strekking'
    elif 'plan' in k.lower() or 'verordening' in k.lower():
        return 'Besluit van Algemene Strekking'
    elif 'melding' in k.lower():
        return 'Melding'
    elif k in ['vergadering of stuk van het bestuur', 'verkiezingen', 'andere voorlichtingsinformatie', 'inspraak', 'openingstijden', 'afvalinzameling']:
        return 'Voorlichting'
    else:
        return 'Overige'
        

In [73]:
df['Informatietypen'] = df['tooiwl_rubriek'].map(rubrieken)

Hier is dus gelijk het onderverdelen in de verschillende informatietypen. 

In [74]:
df['Informatietypen'].value_counts()

Informatietypen
Beschikking                       104719
Besluit van Algemene Strekking      9039
Overige                             5948
Melding                             1714
Voorlichting                         318
Name: count, dtype: int64

Hieruit blijkt wel dat het grootste deel van de publicaties bestaan uit beschikkingen, wat ook wel verwacht kon worden. Gevolgd door de besluiten van algemene strekkingen. Dit zijn twee losstaande informatiecategorieen die volgens de Woo moeten worden gepubliceerd. Dus ik had ook wel verwacht dat deze het vaakst voor zouden komen.

De rest van de analyse wordt gedaan met de beschikkingen omdat deze het meest van belang zijn.

In [75]:
df=df[df['Informatietypen'] == 'Beschikking']

In [76]:
# Een overzicht van alle kolommen die we hebben
print(df.columns)
print('')
print(df['dc_date_year'].value_counts())

Index(['dc_title', 'dc_date_year', 'foi_publishedDate', 'dc_source',
       'tooiwl_topic', 'dc_publisher_name', 'foi_startDate', 'foi_endDate',
       'tooiwl_rubriek', 'dc_type', 'dc_publisher', 'dc_description',
       'foi_files', 'foi_betreftRegeling', 'tooiwl_rubriek_code',
       'OVERHEIDop.gebiedsmarkering', 'kml_Point', 'kml_coordinates',
       'OVERHEIDop.ligtInGemeente', 'OVERHEIDop.huisnummer',
       'OVERHEIDop.postcode', 'OVERHEIDop.straatnaam', 'OVERHEIDop.woonplaats',
       'OVERHEIDop.huisletter', 'kml_LineString', 'OVERHEIDop.geometrieLabel',
       'kml_Polygon', 'OVERHEIDop.waterschapsnaam',
       'OVERHEIDop.huisnummerToevoeging', 'OVERHEIDop.effectgebied',
       'Test_Terinlagelegging', 'OVERHEIDop.kadastraleSectie',
       'OVERHEIDop.perceelnummer', 'Informatietypen'],
      dtype='object')

dc_date_year
2021    13588
2020    12791
2022    12159
2019    11638
2018    10841
2023    10542
2017     9978
2016     7791
2015     6545
2014     6031
2013     2815


In [77]:
grouped_data = df.groupby(['dc_date_year', 'dc_publisher_name'])['dc_title'].count().reset_index()

pivot_table = pd.pivot_table(grouped_data, values='dc_title', index='dc_date_year', columns='dc_publisher_name', fill_value=0).astype(int).sort_index(ascending=False).T
pivot_table['Totaal'] = pivot_table.sum(axis=1)
pivot_table.loc['Aantal per Jaar'] = pivot_table.sum(axis=0)

display(pivot_table)

dc_date_year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,Totaal
dc_publisher_name,,,,,,,,,,,,
Hoogheemraadschap De Stichtse Rijnlanden,714,965,874,639,703,1094,938,777,789,745,73,8311
Hoogheemraadschap Hollands Noorderkwartier,1125,1443,1897,1970,2037,1586,1647,1292,1428,1253,0,15678
Hoogheemraadschap van Delfland,1302,968,1797,1210,967,843,736,798,587,0,0,9208
Hoogheemraadschap van Rijnland,644,806,745,797,630,725,781,705,805,1004,705,8347
Hoogheemraadschap van Schieland en de Krimpenerwaard,552,756,730,775,540,485,553,578,34,0,1,5004
Waterschap Aa en Maas,282,295,382,288,246,161,150,11,0,0,0,1815
"Waterschap Amstel, Gooi en Vecht",489,562,663,681,854,720,572,0,0,0,0,4541
Waterschap Brabantse Delta,698,914,1455,2291,1779,1684,905,614,485,869,842,12536
Waterschap De Dommel,373,230,227,254,219,147,140,3,0,0,0,1593


Als eerste tabel volgen de tellingen van het aantal beschikkingen per waterschap per jaar. Zoals meestal het geval is zijn bepaalde organisaties eerder begonnen met het publiceren. Dit kan hier door twee redenen komen: 1. Het waterschap is pas in de recentste jaren opgericht (bijvoorbeeld Waterschap Limburg wat op 1 januari 2017 is ontstaan) en daarna is altijd gepubliceerd. 2. Het waterschap is pas later begonnen met publiceren. Zoals bijvoorbeeld Hunze en Aa's en Amstel, Gooi en Vecht. 

Zoals verwacht neemt het aantal beschikkingen met de jaren toe. Tot dat het rond 2021 stabiliseerd. Waarschijnlijk worden vanaf dit moment alle mogelijke beschikkingen ook echt openbaar gemaakt en is het getal hierna vooral afhankelijk van het aantal daadwerkelijk bestaande documenten. 

**Opvallende zaken individuele waterschappen**
- Scheldestromen: Hier is men eigenlijk pas begonnen met het publiceren van beschikkingen in 2021. Hiervoor zijn er wel een aantal zeldzame beschikkingen gepubliceerd, maar zeker niet in grote getalen. 

In [78]:
grouped_data = pd.DataFrame(df.groupby(['dc_publisher_name'])['foi_files'].sum())
grouped_data['totaal'] = pivot_table['Totaal']
grouped_data.loc['Totaal alle organisaties'] = grouped_data.sum(axis=0)
grouped_data['bijlagen'] = grouped_data['foi_files']-grouped_data['totaal']
grouped_data['files/org'] = grouped_data['bijlagen']/grouped_data['totaal']

display(grouped_data)

,foi_files,totaal,bijlagen,files/org
dc_publisher_name,,,,
Hoogheemraadschap De Stichtse Rijnlanden,16334,8311,8023,0.965347
Hoogheemraadschap Hollands Noorderkwartier,15720,15678,42,0.002679
Hoogheemraadschap van Delfland,18912,9208,9704,1.053866
Hoogheemraadschap van Rijnland,13992,8347,5645,0.676291
Hoogheemraadschap van Schieland en de Krimpenerwaard,5004,5004,0,0.0
Waterschap Aa en Maas,1816,1815,1,0.000551
"Waterschap Amstel, Gooi en Vecht",16197,4541,11656,2.566835
Waterschap Brabantse Delta,27488,12536,14952,1.192725
Waterschap De Dommel,2274,1593,681,0.427495


De de volgende tabel staan het aantal documenten per organisatie en daarmee het gemiddeld aantal documenten per beschikking weergegeven. Hierbij moet worden meegerekent dat er altijd minstens 1 document aanwezig is dat geen bijlage is. Hierin valt op dat er grote verschillen zijn in de mate van het publiceren van bijlagen. Een aantal organisaties lijken hiermee heel consequent te zijn zoals Waterschap Limburg en Waterschap AGV die gemiddeld meerdere bijlagen per beschikking hebben. Terwijl er ook organisaties zijn waarbij een bijlage vrij zelfdzaam is zoals bijvoorbeeld Scheldestromen en Rivierenland. Het grootste deel van de waterschappen zit er een beetje tussen in wat ook wel verklaard dat er gemiddeld 0.66 bijlagen zijn per publicatie. En bij een beschikking van Schieland en Krimpenewaard zit zelfs nooit een bijlage, en bij Aa en Maas en Fryslan is dit slechts een enkele keer het geval.

Het is echter niet waarschijnlijk dat dit betekent dat bijna 66% van de beschikkingen bijlagen heeft. Waarschijnlijk is dit een veel lager percentage. 

In [79]:
foi_files_counter = dict(df['foi_files'].value_counts())
print(foi_files_counter)

# naast het veranderen van counter in value_counts moest ook de 1 worden toegevoegd omdat in Maik's model altijd 1 document is.
print(1-(foi_files_counter[1] / sum(foi_files_counter.values())))

{1: 62625, 2: 30190, 3: 5990, 4: 2718, 5: 1568, 6: 654, 7: 346, 8: 171, 9: 119, 10: 86, 11: 57, 12: 40, 13: 26, 14: 25, 17: 14, 15: 13, 16: 11, 18: 6, 23: 6, 27: 5, 26: 5, 20: 5, 19: 5, 21: 4, 22: 4, 24: 4, 35: 3, 37: 2, 66: 2, 28: 2, 32: 2, 63: 2, 34: 1, 174: 1, 25: 1, 51: 1, 54: 1, 29: 1, 31: 1, 30: 1, 482: 1}
0.40197098902777906


Door het uitvoeren van de pandas value_counts kan worden berekend hoeveel procent dit daadwerkelijk is. Het blijkt dus dat dit inderdaad een stuk meer is dan het totaal zou laten zien. Het blijkt dat slechts 40 procent van de beschikkingen maar minimaal 1 bijlage heeft. 

Wat ik zelf ook nog interessant vind om naar te kijken zijn extreme waarden. Zijn er bijvoorbeeld publicaties met een extreem hoog aantal bijlagen? Natuurlijk kan dit ook al blijken uit de counter.

In [80]:
print(f"{len(df[df['foi_files'] > 5])} publicaties met meer dan 5 bijlagen.")
print(f"{len(df[df['foi_files'] > 10])} publicaties met meer dan 10 bijlagen.")
print(f"{len(df[df['foi_files'] > 25])} publicaties met meer dan 25 bijlagen.")
print(f"{len(df[df['foi_files'] > 50])} publicaties met meer dan 50 bijlagen.")
print(f"{len(df[df['foi_files'] > 100])} publicaties met meer dan 100 bijlagen.")

result = df[df['foi_files'] > 480]
for i in result.dc_source:
    print(i)

1628 publicaties met meer dan 5 bijlagen.
252 publicaties met meer dan 10 bijlagen.
31 publicaties met meer dan 25 bijlagen.
8 publicaties met meer dan 50 bijlagen.
2 publicaties met meer dan 100 bijlagen.
https://zoek.officielebekendmakingen.nl/wsb-2023-3911


Het blijkt dus dat het vrij zeldzaam is voor een beschikking om meer dan 5 bijlagen te hebben. Dit is in flink minder dan 2 procent van de gevallen zo. En het worden er seeds minder naarmate het aantal bijlagen toeneemt. Toch zijn er nog 8 beschikkingen met meer dan 50 bijlagen. En is er zelfs een extreme outlier met 481 bijlagen. De meeste van deze outliers komen van het waterschap brabantse delta. Al is dit wel één van de waterschappen met de meeste publicaties. 

In [93]:
grouped_data = pd.DataFrame(df.groupby(['dc_date_year'])['foi_files'].sum())
grouped_data['totaal'] = df.groupby(['dc_date_year'])['foi_files'].count()

grouped_data.loc['Totaal alle jaren'] = grouped_data.sum(axis=0)
grouped_data['bijlagen'] = grouped_data['foi_files']-grouped_data['totaal']
grouped_data['files/org'] = grouped_data['bijlagen']/grouped_data['totaal']

display(grouped_data)

,foi_files,totaal,bijlagen,files/org
dc_date_year,,,,
2013,3662,2815,847,0.300888
2014,7976,6031,1945,0.3225
2015,9674,6545,3129,0.478075
2016,11460,7791,3669,0.470928
2017,17039,9978,7061,0.707657
2018,21503,10841,10662,0.983489
2019,21347,11638,9709,0.83425
2020,24724,12791,11933,0.932922
2021,24588,13588,11000,0.809538


Hier is het aantal bijlagen per beschikking gegroepeerd per jaar. Het valt hier op dat in de eerste vier jaren de waarde gemiddeld rond de 0.3 tot 0.5 bijlagen per beschikking lag, om vervolgens vanaf 2017 te stijgen naar bijna 1 bijlage per beschikking en dit werd volgehouden tot en met 2021. Waarna Het aantal bijlagen aanzienlijk is gedaald naar een vergelijkbaar niveau als de eerdere jaren. Dit lijkt dus geen goede ontwikkeling te zijn. Wellicht dat de terinzageleggingen hiervan de oorzaak zijn en dat de organisaties denken dat de bijlagen niet meer gepubliceerd moeten worden als ze al in de terinzagelegging komen. Met als nadeel natuurlijk dat de documenten maar voor een paar maanden terug te vinden zijn.

Edit: Bij de waterschappen zijn bijna geen terinzageleggingen. Dus ik noem het nalatigheid! (het zijn er wellicht 20 in totaal)


In [82]:
GEO_groupby = df.groupby(['dc_publisher_name'])[['kml_Point', 'kml_Polygon', 'kml_LineString']].sum()
GEO_groupby.loc['Totaal aantal per type'] = GEO_groupby.sum(axis=0)
display(GEO_groupby)

,kml_Point,kml_Polygon,kml_LineString
dc_publisher_name,,,
Hoogheemraadschap De Stichtse Rijnlanden,2000,245,394
Hoogheemraadschap Hollands Noorderkwartier,4862,13,4
Hoogheemraadschap van Delfland,3530,8,260
Hoogheemraadschap van Rijnland,1400,64,358
Hoogheemraadschap van Schieland en de Krimpenerwaard,1004,692,1515
Waterschap Aa en Maas,795,55,46
"Waterschap Amstel, Gooi en Vecht",1872,2,13
Waterschap Brabantse Delta,1979,70,695
Waterschap De Dommel,841,57,14


Dit is het aantal keer dat een bepaald soort geolocatie voorkomt per organisatie. Het meest voorkomende soort geolocatie is een point, daarna een linestring en daarna een polygon. Eigenlijk vrij logisch dus. Het valt wel op dat een aantal organisaties wel een iets afwijkende verdeling hebben. Bijvoorbeeld wetterskip fryslan heeft verwassend veel polygons en linestrings vergeleken met het gemiddelde. Hetzelfde geld ook wel voor Rijn en IJssel en Schieland en de Krimpenewaard.

Deze getallen vallen natuurlijk veel lager uit dan het totaal aantal beschikkingen. Dit komt omdat alleen het nieuwe geo-systeem is meegenomen. Vandaar dat het geen enkel nut heeft om te groeperen voor jaren voor 2021. 

In [83]:
GEO_groupby = df.groupby(['dc_date_year'])[['OVERHEIDop.gebiedsmarkering', 'kml_Point',
       'kml_coordinates', 'OVERHEIDop.ligtInGemeente', 'OVERHEIDop.huisnummer',
       'OVERHEIDop.postcode', 'OVERHEIDop.straatnaam', 'OVERHEIDop.woonplaats',
       'OVERHEIDop.huisletter', 'kml_LineString', 'OVERHEIDop.geometrieLabel',
       'kml_Polygon', 'OVERHEIDop.waterschapsnaam',
       'OVERHEIDop.huisnummerToevoeging', 'OVERHEIDop.effectgebied',
       'OVERHEIDop.kadastraleSectie', 'OVERHEIDop.perceelnummer']].sum()

GEO_groupby.index = pd.to_numeric(GEO_groupby.index)
GEO_groupby = GEO_groupby[GEO_groupby.index >= 2021]

display(GEO_groupby.T)

dc_date_year,2021,2022,2023
OVERHEIDop.gebiedsmarkering,7610,14630,16803
kml_Point,6332,11403,13160
kml_coordinates,6332,11403,13160
OVERHEIDop.ligtInGemeente,3992,7324,6598
OVERHEIDop.huisnummer,2891,5140,4907
OVERHEIDop.postcode,2855,5076,4853
OVERHEIDop.straatnaam,2891,5140,4907
OVERHEIDop.woonplaats,2891,5140,4907
OVERHEIDop.huisletter,364,682,767
kml_LineString,979,2319,2444


Hier een overzicht van alle Geo attributen die verzameld kunnen worden. Het grootste deel hiervan zijn dingen die we mogen verwachten zoals adressgegevens. Maar er zijn ook een aantal obscure attributen: Dit zijn effectgebied, kadastraleSectie en perceelnummer (waarbij de laatste twee dan weer bij elkaar horen). Hetzelfde geld eigenlijk ook voor huisnummer, straatnaam en woonplaats. Deze horen ook altijd bij elkaar. 

Ik heb alleen geen idee wat het verschil tussen huisletter en huisnummerToevoeging is dus dit zal ik nog onderzoeken! **Op basis van wat ik weet is het niet hetzelfde, maar ik heb ook niet echt kunnen ontdekken wat de toevoeging precies betekent!**

In [84]:
result = df[df['OVERHEIDop.gebiedsmarkering'] > 0]
result1 = result.groupby('dc_publisher_name')['OVERHEIDop.gebiedsmarkering'].sum()
result2 = result.groupby('dc_publisher_name')['OVERHEIDop.gebiedsmarkering'].count()
res_df = pd.DataFrame({'Aantal Gebiedsmarkeringen': result1, 'Aantal Publicaties': result2})
res_df['Geo per Publicatie'] = res_df['Aantal Gebiedsmarkeringen'] / res_df['Aantal Publicaties']
display(res_df)

,Aantal Gebiedsmarkeringen,Aantal Publicaties,Geo per Publicatie
dc_publisher_name,,,
Hoogheemraadschap De Stichtse Rijnlanden,2639,2189,1.205573
Hoogheemraadschap Hollands Noorderkwartier,4882,3496,1.396453
Hoogheemraadschap van Delfland,3798,2742,1.38512
Hoogheemraadschap van Rijnland,1823,1801,1.012215
Hoogheemraadschap van Schieland en de Krimpenerwaard,3212,1637,1.962126
Waterschap Aa en Maas,896,762,1.175853
"Waterschap Amstel, Gooi en Vecht",1888,1360,1.388235
Waterschap Brabantse Delta,2745,2242,1.224353
Waterschap De Dommel,912,729,1.251029


Het valt dus op dat het aantal geolocaties per publicatie (voor de publicaties met minimaal 1 geolocatie) altijd lager ligt dan 2 (behalve zuiderzeeland. Al zijn er ook organisaties die bijna nooit meer dan 1 geolocaties hebben zoals Vechtstromen en Noorderzijlvest. Terwijl Zuiderzeeland en Wetterskip Fryslan rond de twee geolocaties hebben. Dit is eigenlijk best vreemd want je zou dan verwachten dat de waterschappen met weinig geolocaties dan vaker voor een linestring of polygon zouden kiezen, maar dit blijkt niet het geval te zijn want dit komt juist veel vaker voor bij wetterskip fryslan. 

Omdat geolocaties een oneindig lange lijst kunnen worden ben ik ook nog geinteresseerd in de extreme waardes hier. Zijn er publicaties met een heel hoog aantal geolocaties?

In [85]:
foi_files_counter = dict(df['OVERHEIDop.gebiedsmarkering'].fillna(0).value_counts())
print(foi_files_counter)

print(foi_files_counter[0] / sum(foi_files_counter.values()))

{0: 75540, 1: 24454, 2: 2867, 3: 898, 4: 466, 5: 125, 6: 74, 10: 59, 7: 50, 8: 35, 9: 33, 12: 22, 13: 21, 11: 19, 17: 14, 14: 12, 15: 10, 16: 5, 18: 4, 25: 4, 20: 3, 23: 1, 28: 1, 22: 1, 24: 1}
0.7213590656900849


In [94]:
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 5])} publicaties met meer dan 5 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 10])} publicaties met meer dan 10 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 25])} publicaties met meer dan 25 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 50])} publicaties met meer dan 50 locaties.")
print(f"{len(df[df['OVERHEIDop.gebiedsmarkering'] > 100])} publicaties met meer dan 100 locaties.")

result = df[df['OVERHEIDop.gebiedsmarkering'] > 25]
for i in result.dc_source:
    print(i)

369 publicaties met meer dan 5 locaties.
118 publicaties met meer dan 10 locaties.
1 publicaties met meer dan 25 locaties.
0 publicaties met meer dan 50 locaties.
0 publicaties met meer dan 100 locaties.
https://zoek.officielebekendmakingen.nl/wsb-2023-11865


Het is hier toch wel vrij zeldzaam dat een publicatie meer dan 5 geolocaties heeft. Dit gebeurt eigenlijk maar in minder dan een fractie van de gevallen. En meer dan 25 locaties is al helemaal zeldzaam. Dit komt bij de beschikkingen zelfs slechts 1 keer voor. Bij andere informatiecategorieën komen wel vaker extreme waarden voor.

In [95]:
dict(df['tooiwl_rubriek_code'].fillna(0).value_counts())[0] / sum(df['tooiwl_rubriek_code'].fillna(0).value_counts())

0.7213590656900849

Helaas is de tooi_rubriek code niet echt rendabel om te gebruiken wegens het grote aantal rubrieken zonder tooi code. In 72% van de gevallen ontbreekt een code. Hoewel ik dus vermoedde dat dit wel zou kunnen voor meer recentere gevallen. Dat ga ik nu onderzoeken!

In [88]:
result_rubriek = df.groupby('dc_date_year')['tooiwl_rubriek_code'].count()

result_gebied = df[df['OVERHEIDop.gebiedsmarkering'] != np.nan]
result_gebied = result_gebied.groupby('dc_date_year')['OVERHEIDop.gebiedsmarkering'].count()

result_df = pd.DataFrame({'TOOI_rubriek_code': result_rubriek, 'GeoLocaties': result_gebied})
display(result_df)

,TOOI_rubriek_code,GeoLocaties
dc_date_year,,
2013,0,0
2014,0,0
2015,0,0
2016,0,0
2017,0,0
2018,0,0
2019,0,0
2020,0,0
2021,6478,6478


Tijdens dit onderzoekje bleek ten eerste dat dit ook echt klopt dat dus in de laatste paar jaar dit wel consitent werkt met deze TOOI. Maar verder viel mij nog wat op. Het aantal documenten met de nieuwe TOOI bleek nogal overeen te komen met het patroon wat ik had gevonden voor de geolocaties. En toen ik deze erbij haalde bleek dit zelfs een honder procent match te zijn. Dit kan geen toeval zijn en het lijkt dus dat deze veranderingen tegelijkertijd zijn ingevoerd!

In [89]:
rubriek_counter = df['tooiwl_rubriek'].value_counts()
display(rubriek_counter)

tooiwl_rubriek
Beschikkingen | afhandeling                               62188
Beschikkingen | aanvraag                                  13352
vergunning voor activiteiten op of in oppervlaktewater    11103
andere vergunning                                         10330
omgevingsvergunning                                        4962
andere beschikking                                         2771
exploitatievergunning                                         7
evenementenvergunning                                         6
Name: count, dtype: int64

Bij de rubrieken van de beschikkingen valt op dat de gestandaardiseerde rubrieknamen van het oude model het vaakst voorkomen. Terwijl het nieuwe model veel meer in detail gaat wat voor beschikking het exact is. Exploitatie en evenementenvergunning zijn wel heel zeldzaam bij de waterschappen, maar dit is ook iets wat ik meer verwacht bij gemeenten en wellicht provincies.

In [96]:
rubriek_counter = df['tooiwl_topic'].value_counts()
display(rubriek_counter)

tooiwl_topic
Ruimte en infrastructuur | Organisatie en beleid           55687
Natuur en milieu | Organisatie en beleid                   17716
Bestuur | Organisatie en beleid                            14413
Ruimte en infrastructuur | Waterkeringen en waterbeheer     7724
Bestuur | Waterschappen                                     5418
Natuur en milieu | Water                                    1357
Verkeer | Organisatie en beleid                              932
Recht | Organisatie en beleid                                675
Cultuur en recreatie | Organisatie en beleid                 588
Recht | Bezwaar en klachten                                   58
Natuur en milieu | Stoffen                                    45
Sociale zekerheid | Organisatie en beleid                     22
Migratie en integratie | Organisatie en beleid                20
Openbare orde en veiligheid | Organisatie en beleid           17
Huisvesting | Organisatie en beleid                           10
Onderwijs en

Dit is een counter van de tooiwl_topics. Je ziet dus dat er een aantal heel vaak voorkomen, maar dat een vrij groot deel extreem zeldzaam is. Hoewel ik hierdoor ook een fout heb ontdekt in mijn json. Want volgens mij kunnen publicaties meerdere van deze topics hebben, maar heb ik er maximaal 1. Dus dit zal ik nog aanpassen in de toekomst! Hoewel ik nog niet precies weet hoe dit geimplementeerd moet worden.

Omdat ik de startdate en enddate nogal vaag vind ga ik deze ook nog even kort bekijken. Met de `foi_betreftRegeling` want daar horen ze volgens mij bij!

In [91]:
res_dates = df.groupby('dc_date_year')[['foi_startDate', 'foi_endDate', 'foi_betreftRegeling']].count()
display(res_dates)

,foi_startDate,foi_endDate,foi_betreftRegeling
dc_date_year,,,
2013,1241,1135,0
2014,2153,2132,0
2015,2518,2500,0
2016,0,0,0
2017,0,0,0
2018,0,0,0
2019,0,0,0
2020,0,0,0
2021,0,0,0


Bij de beschikkingen is er alleen een startdatum en een einddatum in de eerste drie jaar. Ik vermoed dat dit om de reactietermijn gaat. Maar eigenlijk weet ik dat niet zeker. 

In [92]:
result_end_date = df[pd.notnull(df['foi_endDate'])]

# Calculate the time delta in days
result_end_date['time_delta'] = (pd.to_datetime(result_end_date['foi_endDate']) - pd.to_datetime(result_end_date['foi_startDate'])).dt.days

result_end_date.groupby('dc_date_year')['time_delta'].mean()

C:\Users\ramon\AppData\Local\Temp\ipykernel_13664\3340103691.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_end_date['time_delta'] = (pd.to_datetime(result_end_date['foi_endDate']) - pd.to_datetime(result_end_date['foi_startDate'])).dt.days


dc_date_year
2013    37.024691
2014    39.902967
2015    41.461970
Name: time_delta, dtype: float64

Het gaat over gemiddeld 40 dagen dus dat komt wel redelijk overeen met die reactietermijn.